In [ ]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import time
colwidth = 9/2.54  # 9 cm
plt.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['text.usetex'] = False
sns.set(style="ticks")
# plt.rc('font', family='serif')
# plt.rc('font', serif='Times New Roman')

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

# sns.set_style('ticks', {'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_palette("muted")

plot function for C(x):

In [ ]:

def plotit(ts, qp_cs, oracle_cs, gp_cs, names, c_cdot, plot_path):
    plt.figure(figsize=(6,4))
    sns.lineplot(
        x=ts,
        y=qp_cs[c_cdot],
        linestyle="dotted",
        color="black",
        label="nominal",
    )
    sns.lineplot(
        x=ts,
        y=oracle_cs[c_cdot],
        linestyle="dashdot",
        color="black",
        label="oracle",
    )
    for model_cs, name in zip(gp_cs, names):
        sns.lineplot(
            x=ts,
            y=model_cs[c_cdot],
            label=name,
            alpha=0.5,
        )
    plt.xlabel("time(s)")
    plt.ylabel('$C(x)$')
    plt.tight_layout()
    plt.legend()
    # plt.figure(figsize=(1.5,2))
    plt.savefig(plot_path, figsize=(6,4), dpi=300)
    plt.show()
    plt.close()  

In [ ]:
plot C(x):

In [ ]:
import numpy as np
from plant_factory import ControllerFactory
from swift_control.eval import eval_cs
swift_path = "/share/dean/fast_control/models/swift_grid/"
file_path = swift_path + "warm_episodic/linear_static_cost/"
plant_conf = swift_path + "base_config.toml"
plot_path = swift_path + "paper/"
qp_cs = np.load(file_path+"qp_cs.npy")
x_0 = np.array([2.0, 0.0, 0.0, 0.0])
plant = ControllerFactory(plant_conf)
kwargs = x_0, plant.episodic_T, plant.episodic_num_steps
oracle_cs, ts = eval_cs(plant.system, plant.oracle_controller, *kwargs)
gp_cs =  np.load(file_path+"gp_cs.npy")
names = ['AD-RF','AD-K','ADP-RF','ADP-K']
ts = np.linspace(0,10,100)
plotit(ts, qp_cs, oracle_cs, gp_cs, names, 0, plot_path)

plot function for dimension test:

In [ ]:
def plot(data, name, train_time, test_time, gp_names):
    cutoff = 40
    sns.color_palette("Set2",4)
    plot_path = "/share/dean/fast_control/models/swift_grid/paper/"
    steps = data.shape[1]
    pre_xs = np.arange(100, steps * 101, 100)[:cutoff]
    xs = np.tile(pre_xs, 10)

    data = data[:, :cutoff, :]
    train_time = train_time[:, :cutoff]
    plt.figure(figsize=(6,4))
    ax = sns.lineplot(
        x=xs,
        y=data[0, :, :].flatten("F"),
        estimator=np.median,
        errorbar=lambda x: (np.quantile(x, 0.25), np.quantile(x, 0.75)),
        label=gp_names[0],
    )
   
    sns.lineplot(
        x=xs,
        y=data[2, :, :].flatten("F"),
        estimator=np.median,
        errorbar=lambda x: (np.quantile(x, 0.25), np.quantile(x, 0.75)),
        label=gp_names[2],
    )

    sns.lineplot(x=xs, y=data[1, :, :].flatten("F"), label=gp_names[1], alpha=0.6)
    sns.lineplot(x=xs, y=data[3, :, :].flatten("F"), label=gp_names[3], alpha=0.6)

    # ax2 = plt.twinx()
    # for i in range(4):
    #     sns.lineplot(x=pre_xs, y=train_time[i,:30], ax=ax2)
    # sns.lineplot(x=xs, y=test_time, ax=ax2, label="test_time")
    
    # ax2.set_ylabel("train time(s)")
    plt.xlabel("feature dimension")
    ax.set_ylabel(f"{name}")
    plt.tight_layout()
    plt.legend()
    plt.savefig(plot_path + f"{name}_grid.png",dpi=300)
    plt.show()

plot function for training time


def plottof(train_time, names, steps):
    cutoff = 40
    plot_path = "/share/dean/fast_control/models/swift_grid/paper/"
    pre_xs = np.arange(100, steps * 101, 100)[:cutoff]
    print(pre_xs.shape)
    print(train_time.shape)
    plt.figure(figsize=(6,4))
    for i in {0,2}:
        sns.lineplot(x=pre_xs, y=train_time[i,:cutoff], label=names[i])
    for i in {1,3}:
        sns.lineplot(x=pre_xs, y=train_time[i,:cutoff], label=names[i])
    plt.xlabel("feature dimension")
    plt.ylabel("average train time(s)")
    plt.savefig(plot_path + f"train_time.png",dpi=300)
    plt.show()

plot dimenesion test and train time:

In [ ]:
import numpy as np
static_path = "/share/dean/fast_control/models/swift_grid/static_tests/small/"
names = ['AD-RF','AD-K','ADP-RF','ADP-K']
rmse = np.load(static_path + "static_tests_rmse_grid.npy")
train_time = np.load(static_path + "static_tests_train_time_grid.npy")
test_time = np.load(static_path + "static_tests_train_time_grid.npy")
plot(rmse, "RMSE", train_time.mean(axis=2), test_time, names)
plottof(train_time.mean(axis=2), names, rmse.shape[1])

In [ ]:
import numpy as np
static_path = "/share/dean/fast_control/models/swift_grid/static_tests/medium/"
names = ['AD-RF','AD-K','ADP-RF','ADP-K']
rmse = np.load(static_path + "static_tests_rmse_grid.npy")
train_time = np.load(static_path + "static_tests_train_time_grid.npy")
test_time = np.load(static_path + "static_tests_train_time_grid.npy")
plot(rmse, "RMSE", train_time, test_time, names)
plottof(train_time, names, rmse.shape[1])